任意一个图像编辑器都能对图像进行一定程度的操作，如放大缩小，旋转等。而这些涉及区域中的空间变化，这就是几何变换。要做到这些：
1.算法实现空间坐标变换；
2.插值算法完成输出图像每个像素的灰度值

# 仿射变换(affine transformation)

## 不同操作的仿射矩阵定义

仿射变换作为一种比较常见的变换，这里首先介绍一下。其在二维空间的坐标表示：
$$\left[\begin{array} {cccc}
X \\
Y \\
\end{array} \right]
= 
\left[\begin{array} {cccc}
a_11 & a_12\\
a_21 & a_22\\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
\end{array} \right]
+
\left[\begin{array} {cccc}
a_13 \\
a_23 \\
\end{array} \right]
$$ 


为了表达的简单，在原坐标的基础上，引入第三个数值 1 的坐标，变化为齐次坐标，表达如下：
$$\left[\begin{array} {cccc}
X \\
Y \\
1 \\
\end{array} \right]
= 
A
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]
$$ 
其中 A为仿射矩阵，如下：
$$
A
= 
\left[\begin{array} {cccc}
a_11 & a_12 & a_13 \\
a_21 & a_22 & a_23 \\
0   & 0   & 1   \\
\end{array} \right]
$$ 

其中比较简单的操作就包括，平移，缩放，旋转。

平移：沿坐标轴将对应点的坐标进行移动，变换前后图像大小不发生改变，只是坐标点发生变化。其变换矩阵：
$$\left[\begin{array} {cccc}
X \\
Y \\
1\\
\end{array} \right]
= 
\left[\begin{array} {cccc}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1  \\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]$$ 

缩放：在二维空间坐标上的表现，沿坐标轴将对应点的坐标进行缩放，其意义是，相对于变换的中心坐标，将坐标点距离中心坐标点的距离横坐标缩放Sx，纵坐标缩放Sy. 变换前后图像大小不发生变化，同时坐标点发生变化。其变换矩阵(中心坐标为（0,0）)：
$$\left[\begin{array} {cccc}
X \\
Y \\
1\\
\end{array} \right]
= 
\left[\begin{array} {cccc}
S_x  & 0   & 0 \\
0   & S_y  & 0 \\
0   & 0   & 1 \\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]$$ 
对于非中心坐标为（0，0）的缩放变换，我门可以认为其是先进行平移，然后进行缩放，在进行平移得到，其变换矩阵可以表示为（x0, y0）:
$$\left[\begin{array} {cccc}
X \\
Y \\
1\\
\end{array} \right]
= 
\left[\begin{array} {cccc}
0   & 0   & x_0 \\
0   & 0   & y_0 \\
0   & 0   & 1  \\
\end{array} \right]
\left[\begin{array} {cccc}
S_x  & 0   & 0 \\
0   & S_y  & 0 \\
0   & 0   & 1 \\
\end{array} \right]
\left[\begin{array} {cccc}
0   & 0   & -x_0 \\
0   & 0   & -y_0 \\
0   & 0   & 1   \\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]$$ 
对于缩放图像放大或者缩小所产生的图像会产生

旋转：将坐标绕某一点(x0,yo),绕顺时针转转某一角度(α)。
如图，可计算得出：
$$ \begin{eqnarray*}
\cos \theta & = \frac{x}{p}, \sin \theta & = \frac{y}{p} \\
\end{eqnarray*}$$
其中 p 表示(x,y)到中心点(0，0)的距离，则
$$ \begin{eqnarray*}
\cos (\theta + \alpha) & = \cos \theta \sin \alpha - \sin \theta \cos \alpha = \frac{x}{p} \cos \alpha + \frac{y}{p} \sin \alpha = \frac{X}{p}\\
\sin (\theta + \alpha) & = \sin \theta \cos \alpha - \cos \theta \sin \alpha = \frac{y}{p} \cos \alpha + \frac{x}{p} \sin \alpha = \frac{Y}{p}\\
\end{eqnarray*}$$
化简上面的公式可得：
$$ X = x \cos \alpha - y \sin \alpha , Y = x \sin \alpha + y \cos \alpha $$
由此推到可得其矩阵表达为：
$$\left[\begin{array} {cccc}
X \\
Y \\
1\\
\end{array} \right]
= 
\left[\begin{array} {cccc}
\cos \alpha & -\sin \alpha & 0 \\
\sin \alpha & \cos \alpha  & 0 \\
0        & 0        & 1 \\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]$$ 
推广到一般，绕(x0, y0) 顺时针旋转，可得：
$$\left[\begin{array} {cccc}
X \\
Y \\
1\\
\end{array} \right]
= 
\left[\begin{array} {cccc}
0   & 0   & x_0 \\
0   & 0   & y_0 \\
0   & 0   & 1  \\
\end{array} \right]
\left[\begin{array} {cccc}
\cos \alpha & -\sin \alpha & 0 \\
\sin \alpha & \cos \alpha  & 0 \\
0        & 0        & 1 \\
\end{array} \right]
\left[\begin{array} {cccc}
0   & 0   & -x_0 \\
0   & 0   & -y_0 \\
0   & 0   & 1   \\
\end{array} \right]
\left[\begin{array} {cccc}
x \\
y \\
1 \\
\end{array} \right]$$ 
逆时针类似推导即可。

## 仿射矩阵计算

1.方程法
仿射矩阵有六个未知数，因此给出三对坐标值，即可计算出六个参数。OpenCV提供函数：cv2.getAffineTransform(src, dst),计算仿射矩阵，其中src代表原坐标，dst代表变换之后的坐标，且都是3 * 2 的二维ndarray的浮点型数据类型。

In [8]:
import cv2
import numpy as np
src = np.array([[0,0], [200,0], [0,200]], np.float32)
dst = np.array([[0,0], [100,0], [0,100]], np.float32)
A = cv2.getAffineTransform(src, dst)
A

array([[0.5, 0. , 0. ],
       [0. , 0.5, 0. ]])

其中，OpenCV提供的C++ API则提供了两种方法：

A.使用 Ponit2f保存坐标:

Point2f src[] = {Point2f(0,0), Point2f(200,0), Point2f(0,200)}

Point2f dst[] = {Point2f(0,0), Point2f(100,0), Point2f(0,100)} 

Mat  A = getAffineTransform(src, dst)

其计算的结果 2 * 3 矩阵，数据类型CV_64F。

B.使用 Mat 保存坐标：

Mat src = (Mat_<float>(3,2) << 0, 0, 200, 0, 0, 200)

Mat dst = (Mat_<float>(3,2) << 0, 0, 100, 0, 0, 100)

Mat  A = getAffineTransform(src, dst)

2.矩阵法，我们在已知仿射变换的基本类型，可以通过矩阵点乘实现。例子（缩放两倍，平移）：

In [11]:
import numpy as np
s = np.array([[0.5, 0, 0], [0, 0.5, 0], [0, 0, 1]]) # 缩放
t = np.array([[1, 0, 100], [0, 1, 200], [0, 0, 1]]) # 平移
A = np.dot(t, s)
A

array([[  0.5,   0. , 100. ],
       [  0. ,   0.5, 200. ],
       [  0. ,   0. ,   1. ]])

其中，OpenCV提供的C++ 这是通过“*”和 gemm 函数实现：

Mat src = (Mat_<float>(3,2) << 0, 0, 200, 0, 0, 200)

Mat dst = (Mat_<float>(3,2) << 0, 0, 100, 0, 0, 100)

Mat A

gemm(s, t, 1.0, Mat(), 0, A, 0)

类似可得以(x0, y0) 为中心进行旋转，顺时针旋转 α，其矩阵表达：
$$
A
= 
\left[\begin{array} {cccc}
s_x \cos \alpha & s_y \sin \alpha & (1- s_x \cos \alpha)x_0 - s_y y_0 \sin \alpha \\
-s_x \cos \alpha & s_y \sin \alpha & (1- s_y \cos \alpha)y_0 - s_x x_0 \sin \alpha \\
0   & 0   & 1   \\
\end{array} \right]
$$ 
同理可求得逆时针

In [10]:
import cv2
A = cv2.getRotationMatrix2D((40, 50), 30, 0.5)
A.dtype
A

array([[ 0.4330127 ,  0.25      , 10.17949192],
       [-0.25      ,  0.4330127 , 38.34936491]])

C++的实现：Mat A = getTotationMatrix2D(Point2f(40,50),30, 0.5)

## 插值算法 

算法的目标：是为了将变换之后的所有坐标都能在原始坐标里面找到对应的灰度值。
如图：

插值算法简单的有两种，下面来进行详细介绍：

1.最近邻插值：令变换后像素的灰度值等于距它最近的输入像素的灰度值。

2.双线性插值：在x y两个方向上分别进行一次线性插值，方向先后不影响顺序。